In [1]:
from mpi4py import MPI
import itertools
from sqsgenerator import sqs_optimize, export_structures
from operator import itemgetter
import os
import numpy as np
from sqsgenerator.public import Structure
from pymatgen.core import Structure as PymatgenStructure

# 初始化MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

def generate_combinations(total=1, count=5, step=0.1):
    def find_combinations(current_comb, remaining_total, remaining_count):
        # 检查是否已选择了两个数字且它们的和小于0.5
        if len(current_comb) == 2 and sum(current_comb) < 0.5:
            return  # 如果前两个数字之和小于0.5，则停止递归这个分支
        
        if remaining_count == 0:
            if round(remaining_total, 1) == 0:
                valid_combinations.append(current_comb)
            return

        for value in possible_values:
            if value <= remaining_total:
                find_combinations(current_comb + [value], round(remaining_total - value, 1), remaining_count - 1)

    possible_values = [round(i * step, 1) for i in range(1, int(total / step) + 1)]
    valid_combinations = []
    find_combinations([], total, count)
    
    return valid_combinations

# 示例使用
combinations = generate_combinations(total=1, count=5, step=0.1)
print(combinations)


# 定义元素和比例
ab_elements = ['Mn', 'Fe', 'Co', 'Tc', 'Rh', 'Re', 'Os']
ab_combinations = list(itertools.combinations(ab_elements, 3))

# 生成所有可能的比例组合
valid_combinations = generate_combinations()

# 打印生成的元素组合
if rank == 0:
    print("Generated element combinations:")
    for comb in ab_combinations:
        print(comb)

# 配置 SQS 优化参数并生成结构对象
def generate_sqs_structures(structure_file, supercell, iterations, shell_weights, elements, combinations, ratios):
    # 读取基础结构
    pymatgen_structure = PymatgenStructure.from_file(structure_file)
    base_structure = Structure(lattice=pymatgen_structure.lattice.matrix, 
                               frac_coords=pymatgen_structure.frac_coords, 
                               symbols=[site.species_string for site in pymatgen_structure], 
                               pbc=(True, True, True))
    
    # 找到所有 Ru 原子的索引
    ir_indices = [i for i, symbol in enumerate(base_structure.symbols) if symbol == 'Ir']
    num_ir_atoms = len(ir_indices)

    for ratio in ratios:
        if rank == 0:
            print(f"Processing ratio: {ratio}")
        for combination in combinations:
            if rank == 0:
                print(f"Combination: {combination}")
            total_atoms = 48  # 假设总原子数为144
            o_atoms = 32  # 假设氧原子数为96
            metal_atoms = total_atoms - o_atoms  # 金属原子总数

            composition = {'Ir': int(np.round(ratio[0] * metal_atoms)), 'Ru': int(np.round(ratio[1] * metal_atoms)), 'O': o_atoms}
            for i, elem in enumerate(combination):
                composition[elem] = int(np.round(ratio[i + 2] * metal_atoms))
            
            # 调整以确保总原子数匹配
            total_composition_atoms = sum(composition.values()) - o_atoms
            if total_composition_atoms != metal_atoms:
                # 调整最后一个元素的数量以匹配总原子数
                last_elem = combination[-1]
                composition[last_elem] += metal_atoms - total_composition_atoms
                if composition[last_elem] == 0:
                    if rank == 0:
                        print(f"Skipping combination {combination} with ratio {ratio} due to zero atoms for element {last_elem}.")
                    continue  # 跳过保存该结构
            
            if rank == 0:
                print(f"Combination: {combination}, Ratio: {ratio}, Composition: {composition}")

            # 替换 Ru 原子
            new_species_for_replacement = ['Ir'] * composition['Ir'] + ['Ru'] * composition['Ru']
            for elem in combination:
                new_species_for_replacement += [elem] * composition[elem]
            
            # 确保 new_species_for_replacement 的长度匹配 ru_indices 的长度
            new_species_for_replacement = new_species_for_replacement[:len(ir_indices)]
            
            # 使用 slice_with_species 替换 Ru 原子
            new_structure = base_structure.slice_with_species(new_species_for_replacement, ir_indices)
            
            # 生成 SQS 结构
            configuration = {
                'structure': new_structure,
                'supercell': supercell,
                'iterations': iterations,
                'shell_weights': shell_weights,
                'elements': elements
            }
            
            try:
                results, timings = sqs_optimize(configuration, make_structures=True)
                if rank == 0:
                    # 动态生成文件名
                    output_file = f"Ir{ratio[0]}Ru{ratio[1]}{combination[0]}{ratio[2]}{combination[1]}{ratio[3]}{combination[2]}{ratio[4]}O2"
                    print(f"Exporting structures to file: {output_file}")
                    export_structures(results, functor=itemgetter('structure'), output_file=output_file, writer='pymatgen')
                    print(f"Structures for combination {combination} with ratio {ratio} have been exported to {output_file}.")
            except Exception as e:
                if rank == 0:
                    print(f"Error for combination {combination} with ratio {ratio}: {e}")
                # 这里可以添加更多的错误处理逻辑，例如记录错误日志

# 示例参数
structure_file = 'IrO22.vasp'
supercell = (1, 1, 1)
iterations = int(1e8)
shell_weights = {1: 1.0, 2: 0.5,3: 0.2}  # 添加第二壳层权重

# 调用 generate_sqs_structures 函数
generate_sqs_structures(structure_file, supercell, iterations, shell_weights, ['Ir', 'Ru'], ab_combinations, valid_combinations)

# 终止MPI
MPI.Finalize()

[[0.1, 0.4, 0.1, 0.1, 0.3], [0.1, 0.4, 0.1, 0.2, 0.2], [0.1, 0.4, 0.1, 0.3, 0.1], [0.1, 0.4, 0.2, 0.1, 0.2], [0.1, 0.4, 0.2, 0.2, 0.1], [0.1, 0.4, 0.3, 0.1, 0.1], [0.1, 0.5, 0.1, 0.1, 0.2], [0.1, 0.5, 0.1, 0.2, 0.1], [0.1, 0.5, 0.2, 0.1, 0.1], [0.1, 0.6, 0.1, 0.1, 0.1], [0.2, 0.3, 0.1, 0.1, 0.3], [0.2, 0.3, 0.1, 0.2, 0.2], [0.2, 0.3, 0.1, 0.3, 0.1], [0.2, 0.3, 0.2, 0.1, 0.2], [0.2, 0.3, 0.2, 0.2, 0.1], [0.2, 0.3, 0.3, 0.1, 0.1], [0.2, 0.4, 0.1, 0.1, 0.2], [0.2, 0.4, 0.1, 0.2, 0.1], [0.2, 0.4, 0.2, 0.1, 0.1], [0.2, 0.5, 0.1, 0.1, 0.1], [0.3, 0.2, 0.1, 0.1, 0.3], [0.3, 0.2, 0.1, 0.2, 0.2], [0.3, 0.2, 0.1, 0.3, 0.1], [0.3, 0.2, 0.2, 0.1, 0.2], [0.3, 0.2, 0.2, 0.2, 0.1], [0.3, 0.2, 0.3, 0.1, 0.1], [0.3, 0.3, 0.1, 0.1, 0.2], [0.3, 0.3, 0.1, 0.2, 0.1], [0.3, 0.3, 0.2, 0.1, 0.1], [0.3, 0.4, 0.1, 0.1, 0.1], [0.4, 0.1, 0.1, 0.1, 0.3], [0.4, 0.1, 0.1, 0.2, 0.2], [0.4, 0.1, 0.1, 0.3, 0.1], [0.4, 0.1, 0.2, 0.1, 0.2], [0.4, 0.1, 0.2, 0.2, 0.1], [0.4, 0.1, 0.3, 0.1, 0.1], [0.4, 0.2, 0.1, 0.1, 0.2], 

In [2]:
import os
from pymatgen.io.cif import CifParser
import pandas as pd
from sqsgenerator.public import sqs_analyse

def analyse_cif_files_in_directory(directory):
    """
    对指定目录中的所有.cif文件进行分析并提取objective值。
    返回包含文件名和objective值的数据框。
    """
    data = []

    # 遍历目录中的所有文件
    for filename in os.listdir(directory):
        if filename.endswith('.cif'):
            cif_file_path = os.path.join(directory, filename)
            parser = CifParser(cif_file_path)
            structures = parser.parse_structures()

            # 调用sqs_analyse函数
            results = sqs_analyse(structures, structure_format='pymatgen')

            # 假设results是一个字典，提取objective值
            for result in results.values():
                objective_value = result.get('objective', None)
                if objective_value is not None:
                    data.append({'Filename': filename, 'Objective': objective_value})

    # 将数据转换为DataFrame
    df = pd.DataFrame(data)
    return df

# 使用示例
directory_path = '/home/haoxw/MEOwork/Calculate_dir/SQS/'
df = analyse_cif_files_in_directory(directory_path)
print(df)


/home/haoxw/anaconda3/envs/sqsgen/lib/python3.9/site-packages/pymatgen/io/cif.py:1196: UserWarning: The default value of primitive was changed from True to False in https://github.com/materialsproject/pymatgen/pull/3419. CifParser now returns the cell in the CIF file as is. If you want the primitive cell, please set primitive=True explicitly.
  warnings.warn(


           Filename  Objective
0     107373442.cif   7.965926
1      86197291.cif  10.085185
2      53823169.cif  12.567407
3     121659148.cif   9.729630
4     134814692.cif   9.840741
...             ...        ...
8877  122649240.cif   9.840741
8878   83395349.cif  10.085185
8879   87580707.cif   7.740000
8880  376447814.cif   7.965926
8881   28984430.cif   8.892000

[8882 rows x 2 columns]


In [3]:
import os
from pymatgen.core import Structure
from pymatgen.io.cif import CifWriter

def read_oxygen_coords_from_vasp(vasp_file):
    """
    从 VASP 文件中读取氧原子的分数坐标。

    参数:
    vasp_file (str): VASP 文件路径。

    返回:
    list of list of float: 氧原子的分数坐标列表。
    """
    structure = Structure.from_file(vasp_file)
    oxygen_coords = [site.frac_coords for site in structure if site.species_string == 'O']
    return oxygen_coords

def add_oxygen_to_cif(directory, oxygen_coords):
    """
    遍历目录下的所有 CIF 文件，并将氧原子的位置信息添加到每个 CIF 文件中。

    参数:
    directory (str): 包含 CIF 文件的目录路径。
    oxygen_coords (list of list of float): 氧原子的分数坐标列表。

    返回:
    None
    """
    for filename in os.listdir(directory):
        if filename.endswith(".cif"):
            file_path = os.path.join(directory, filename)
            
            # 读取 CIF 文件
            structure = Structure.from_file(file_path)
            
            # 添加氧原子
            for coord in oxygen_coords:
                structure.append('O', coord)
            
             # 覆盖原 CIF 文件
            writer = CifWriter(structure)
            writer.write_file(file_path)
            print(f"Modified CIF file saved to: {file_path}")

# 指定 VASP 文件路径
vasp_file = "/home/haoxw/MEOwork/Calculate_dir/SQS/IrO22.vasp"

# 读取氧原子的分数坐标
oxygen_coords = read_oxygen_coords_from_vasp(vasp_file)

# 指定包含 CIF 文件的目录路径
directory = "/home/haoxw/MEOwork/Calculate_dir/SQS"

# 调用函数添加氧原子
add_oxygen_to_cif(directory, oxygen_coords)


Modified CIF file saved to: /home/haoxw/MEOwork/Calculate_dir/SQS/107373442.cif
Modified CIF file saved to: /home/haoxw/MEOwork/Calculate_dir/SQS/86197291.cif
Modified CIF file saved to: /home/haoxw/MEOwork/Calculate_dir/SQS/53823169.cif
Modified CIF file saved to: /home/haoxw/MEOwork/Calculate_dir/SQS/121659148.cif
Modified CIF file saved to: /home/haoxw/MEOwork/Calculate_dir/SQS/134814692.cif
Modified CIF file saved to: /home/haoxw/MEOwork/Calculate_dir/SQS/363145.cif
Modified CIF file saved to: /home/haoxw/MEOwork/Calculate_dir/SQS/644509.cif
Modified CIF file saved to: /home/haoxw/MEOwork/Calculate_dir/SQS/32330852.cif
Modified CIF file saved to: /home/haoxw/MEOwork/Calculate_dir/SQS/44112118.cif
Modified CIF file saved to: /home/haoxw/MEOwork/Calculate_dir/SQS/175910330.cif
Modified CIF file saved to: /home/haoxw/MEOwork/Calculate_dir/SQS/13348818.cif
Modified CIF file saved to: /home/haoxw/MEOwork/Calculate_dir/SQS/264034862.cif
Modified CIF file saved to: /home/haoxw/MEOwork/Cal

In [4]:
import os
from pymatgen.core import Structure
from collections import Counter

def get_element_composition(cif_file):
    try:
        structure = Structure.from_file(cif_file)
        element_counts = Counter([str(site.specie) for site in structure])
        composition_str = ''.join([f"{el}{count}" for el, count in element_counts.items()])
        return composition_str
    except Exception as e:
        print(f"Error reading {cif_file}: {e}")
        return None

def rename_cif_files(directory):
    composition_counts = {}
    for filename in os.listdir(directory):
        if filename.endswith(".cif"):
            file_path = os.path.join(directory, filename)
            composition_str = get_element_composition(file_path)
            if composition_str:
                # 计算新的文件名
                if composition_str in composition_counts:
                    composition_counts[composition_str] += 1
                else:
                    composition_counts[composition_str] = 1
                new_filename = f"{composition_str}_{composition_counts[composition_str]}.cif"
                
                # 重命名文件
                new_file_path = os.path.join(directory, new_filename)
                try:
                    os.rename(file_path, new_file_path)
                    print(f"Renamed: {filename} -> {new_filename}")
                except Exception as e:
                    print(f"Error renaming {filename}: {e}")

# 使用示例
directory_path = "/home/haoxw/MEOwork/Calculate_dir/SQS"  # 替换为你的 CIF 文件目录
rename_cif_files(directory_path)


Renamed: 107373442.cif -> Co2Ir5Os3Ru3Rh3O32_1.cif
Renamed: 86197291.cif -> Mn2Tc2Ir6Ru3Rh3O32_1.cif
Renamed: 53823169.cif -> Fe2Co2Ir6Os1Ru5O32_1.cif
Renamed: 121659148.cif -> Mn2Tc3Ir6Os2Ru3O32_1.cif
Renamed: 134814692.cif -> Fe2Ir2Os3Ru6Rh3O32_1.cif
Renamed: 363145.cif -> Mn2Fe2Ir10Ru2O32_1.cif
Renamed: 644509.cif -> Re2Tc2Ir2Ru10O32_1.cif
Renamed: 32330852.cif -> Co3Ir2Os3Ru6Rh2O32_1.cif
Renamed: 44112118.cif -> Mn3Fe2Co2Ir3Ru6O32_1.cif
Renamed: 175910330.cif -> Mn2Co3Ir3Os3Ru5O32_1.cif
Renamed: 13348818.cif -> Tc2Ir3Os1Ru8Rh2O32_1.cif
Renamed: 264034862.cif -> Re3Tc3Ir5Os2Ru3O32_1.cif
Renamed: 4477400.cif -> Co3Tc2Ir8Os1Ru2O32_1.cif
Renamed: 351241288.cif -> Fe2Re3Tc3Ir3Ru5O32_1.cif
Renamed: 56429490.cif -> Co2Re3Tc2Ir6Ru3O32_1.cif
Renamed: 25247823.cif -> Co2Tc2Ir8Os2Ru2O32_1.cif
Renamed: 182623445.cif -> Co2Re3Ir3Os2Ru6O32_1.cif
Renamed: 7215717.cif -> Fe2Re2Tc3Ir6Ru3O32_1.cif
Renamed: 28004451.cif -> Tc2Ir8Os2Ru2Rh2O32_1.cif
Renamed: 151759951.cif -> Re3Tc2Ir2Ru6Rh3O32_1.cif
Re